In [20]:
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
from optuna.trial import TrialState
import joblib

import statistics
from sklearn.model_selection import KFold
import time
from utils import calculate_metric

from pytorch_tabular.models import GANDALFConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig
)
from pytorch_tabular import TabularModel
import warnings

In [2]:
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
# set random seeds
np.random.seed(0)
torch.manual_seed(0)

In [4]:
INPUT_FILE = "../data/data_removing_na.xlsx"
MODEL_PATH = '../output/transformer'
BATCH_SIZE = 32
PROJECT_NAME = "run/GANDALF_bagging"

DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [5]:
DEVICE

device(type='cuda', index=1)

In [6]:
df = pd.read_excel(INPUT_FILE)

In [7]:
df.head()

,rr1_30,currency,seniorioty_adj,coupon rate,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,...,PD_55_pd,PD_56_pd,PD_57_pd,PD_58_pd,PD_59_pd,PD_60_pd,DTD,NI_Over_TA,Size,defaulted_in_last_6_months
0,0.259908,USD,Senior Subordinated Unsecured,9.000,United States,United States,Consumer Discretionary,Retail & Whsle - Discretionary,E-Commerce Discretionary,Bankruptcy Filing,...,0.396731,0.397453,0.398148,0.398819,0.399467,0.400092,-0.732815,-0.007137,-0.852484,False
1,0.032729,USD,Senior Subordinated Unsecured,5.750,United States,United States,Health Care,Health Care,Health Care Facilities & Svcs,Default Corp Action,...,0.957454,0.957467,0.957480,0.957492,0.957503,0.957514,-1.666262,-0.000286,-1.186347,False
2,0.972400,USD,Unsecured,5.675,South Korea,South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,Default Corp Action,...,0.568169,0.568693,0.569197,0.569682,0.570150,0.570600,-1.853366,0.000191,1.053677,False
3,1.047416,CHF,Unsecured,0.125,South Korea,South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,Default Corp Action,...,0.568169,0.568693,0.569197,0.569682,0.570150,0.570600,-1.853366,0.000191,1.053677,False
4,0.848872,JPY,Unsecured,1.750,Japan,Japan,Industrials,Industrial Products,Electrical Equipment,Bankruptcy Filing,...,0.130285,0.130688,0.131081,0.131465,0.131840,0.132206,-0.768857,-0.028058,-1.946507,False


In [8]:
df.shape

(1725, 165)

In [9]:
feature_list = df.columns
feature_list = feature_list.drop('rr1_30')

In [10]:
feature_list

Index(['currency', 'seniorioty_adj', 'coupon rate', 'domicile_country',
       'exchange_country', 'Industry_sector', 'Industry_group',
       'Industry_subgroup', 'event_type', 'event_type_subcategory_sum',
       ...
       'PD_55_pd', 'PD_56_pd', 'PD_57_pd', 'PD_58_pd', 'PD_59_pd', 'PD_60_pd',
       'DTD', 'NI_Over_TA', 'Size', 'defaulted_in_last_6_months'],
      dtype='object', length=164)

In [11]:
category_features = list(df.select_dtypes(include=['object', 'bool']).columns)
non_category_features = [i for i in feature_list if i not in category_features]

In [12]:
print(len(non_category_features))
print(len(category_features))

153
11


In [13]:
category_features

['currency',
 'seniorioty_adj',
 'domicile_country',
 'exchange_country',
 'Industry_sector',
 'Industry_group',
 'Industry_subgroup',
 'event_type',
 'event_type_subcategory_sum',
 'defaulted_in_last_5_years',
 'defaulted_in_last_6_months']

In [14]:
# split data into training and test set
test_size = 0.25
train, test = train_test_split(df, test_size=test_size, random_state=42)

In [15]:
train.head()

,rr1_30,currency,seniorioty_adj,coupon rate,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,...,PD_55_pd,PD_56_pd,PD_57_pd,PD_58_pd,PD_59_pd,PD_60_pd,DTD,NI_Over_TA,Size,defaulted_in_last_6_months
755,0.082481,USD,Senior Unsecured,7.500,United States,United States,Energy,Oil & Gas,Oil & Gas Producers,Default Corp Action,...,0.691011,0.691382,0.691738,0.692080,0.692408,0.692725,-1.084433,-0.052027,-2.074964,False
588,0.378845,USD,Unsecured,6.000,Indonesia,Indonesia,Materials,Materials,Metals & Mining,Default Corp Action,...,0.183801,0.184996,0.186167,0.187313,0.188437,0.189539,-0.540409,0.017209,0.864692,False
585,0.836149,USD,Senior Secured,11.000,United States,United States,Materials,Materials,Metals & Mining,Default Corp Action,...,0.278774,0.280216,0.281615,0.282972,0.284290,0.285571,0.754647,-0.010395,-0.342209,False
1329,0.987208,USD,Senior Secured,9.125,United States,United States,Energy,Oil & Gas,Oil & Gas Services & Equip,Default Corp Action,...,0.190375,0.191471,0.192544,0.193594,0.194622,0.195628,-0.242080,-0.022618,-2.808528,False
973,1.021458,USD,Senior Secured,9.250,United States,United States,Energy,Oil & Gas,Oil & Gas Producers,Default Corp Action,...,0.149501,0.150930,0.152327,0.153692,0.155027,0.156332,1.138686,0.000033,-0.085154,False


RUNNING THE MODEL

In [16]:
# empty cache first
torch.cuda.empty_cache()

In [25]:
EPOCHS = 100

target_col = 'rr1_30'

data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=non_category_features,
    categorical_cols=category_features,
)

optimizer_config = OptimizerConfig()
experiment_config = ExperimentConfig(project_name=PROJECT_NAME,exp_watch="all",log_target="tensorboard")

trainer_config = TrainerConfig(
    batch_size=BATCH_SIZE,
    max_epochs=EPOCHS,
    early_stopping=None
)


model_config = GANDALFConfig(
    task="regression",
    metrics=["mean_absolute_error", "mean_absolute_percentage_error", "mean_squared_error", "r2_score"],
    gflu_stages=1,
    gflu_feature_init_sparsity=0.7517028756773947,
    gflu_dropout=0.310129872375981,
    learning_rate=0.011357265478429133
)


In [26]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config,
    verbose=True
)

In [ ]:
# kf = KFold(n_splits=5, shuffle=True, random_state=42)


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    bagged_pred_df = tabular_model.bagging_predict(
        cv=3, train=train, test=test, aggregate="mean"
    )

2025-03-26 10:29:04,277 - {pytorch_tabular.tabular_model:2386} - INFO - Running Fold 1/3

2025-03-26 10:29:04,280 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-03-26 10:29:04,284 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

2025-03-26 10:29:04,325 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: GANDALFModel

2025-03-26 10:29:04,386 - {pytorch_tabular.models.gandalf.gandalf:109} - INFO - Data Aware Initialization of T0

...

2025-03-26 10:29:04,413 - {pytorch_tabular.tabular_model:678} - INFO - Training Started

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ GANDALFBackbone  │  343 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │  2.5 K │ train │
│ 2 │ _head            │ Sequential       │    241 │ train │
│ 3 │ loss             │ MSELoss          │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 346 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 346 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 28                                                                                          
Modules in eval mode: 0

Output()

In [ ]:
tabular_model.fit(train=train) #validation=test) #by default, it takes 20% of the data as validation to select the best epoch

Seed set to 42


2025-03-26 08:17:18,619 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-03-26 08:17:18,626 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because

2025-03-26 08:17:18,672 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: GANDALFModel

...

Trainable params: 343 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 343 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 28                                                                                          
Modules in eval mode: 0

Output()

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connec
tor.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the 
value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connec
tor.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the 
value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The 
number of training batches (33) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower 
value for log_every_n_steps if you want to see logs for the training epoch.

In [ ]:
 # save metrics
mae, mape, rmse, rsqr = calculate_metric(bagged_pred_df.to_numpy().squeeze(), test[target_col].to_numpy())
print(f"Test average mean absolute error: {mae}")
print(f"Test average mean absolute percentage error: {mape}")
print(f"Test average root mean squared error: {rmse}")
print(f"Test average R2: {rsqr}")

Test average mean absolute error: 0.21609474190348582
Test average mean absolute percentage error: 362.5249817135827
Test average root mean squared error: 0.2661158489989658
Test average R2: 0.3817428281188312


In [ ]:
# saving model
joblib.dump(tabular_model, f"{PROJECT_NAME}/final_models/final.joblib")

['run/GANDALF/final_models/final.joblib']

5 fold CV

In [ ]:
# Define cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
EPOCHS = 100
val_mae = []
val_mape = []
val_rmse = []
val_rsqr = []

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config,
    verbose=True
)

trainer_config = TrainerConfig(
    batch_size=BATCH_SIZE,
    max_epochs=EPOCHS,
    early_stopping=None
)

model = None
datamodule = None

for index, (train_idx, val_idx) in enumerate(kf.split(df)):
        # Create training and validation datasets for the current fold
        train_fold, val_fold = df.iloc[train_idx], df.iloc[val_idx]
        
        if datamodule is None:
            # Initialize datamodule and model in the first fold
            # uses train data from this fold to fit all transformers
            datamodule = tabular_model.prepare_dataloader(
                train=train_fold, validation=val_fold, seed=42
            )
            model = tabular_model.prepare_model(datamodule)
        else:
            # Creates a copy of the datamodule with same transformers but different train and validation data
            datamodule = datamodule.copy(train=train_fold, validation=val_fold)
            
        # Train the model
        tabular_model.train(model, datamodule)
        pred_df = tabular_model.predict(val_fold)

        # Validation of the model.
        mae, mape, rmse, rsqr = calculate_metric(pred_df.to_numpy().squeeze(), val_fold[target_col].to_numpy())
        val_mae.append(mae)
        val_mape.append(mape)
        val_rmse.append(rmse)
        val_rsqr.append(rsqr)

2025-03-21 16:13:16,769 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-03-21 16:13:16,775 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace meth

2025-03-21 16:13:16,821 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: GANDALFModel

2025-03-21 16:13:16,890 - {pytorch_tabular.models.gandalf.gandalf:109} - INFO - Data Aware Initialization of T0

...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py
:298: The number of training batches (44) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a
lower value for log_every_n_steps if you want to see logs for the training epoch.

`Trainer.fit` stopped: `max_epochs=5` reached.


2025-03-21 16:14:41,448 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-03-21 16:14:41,450 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_tabular/utils/python_utils.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur

In [ ]:
val_rmse

[0.29893204490461195,
 0.28676149441184196,
 0.260360608362095,
 0.252864688137029,
 0.24041399344901898]

In [ ]:
print(f"Test average mean absolute error: {statistics.mean(val_mae)}")
print(f"Test average mean absolute percentage error: {statistics.mean(val_mape)}")
print(f"Test average root mean squared error: {statistics.mean(val_rmse)}")
print(f"Test average R2: {statistics.mean(val_rsqr)}")

Test average mean absolute error: 0.21320731905445486
Test average mean absolute percentage error: 3376.7316955052315
Test average root mean squared error: 0.2678665658529194
Test average R2: 0.34381419201652874
